In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [9]:
x_data = obtain_path(
    df=df_rumours, 
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=20
)

x_data.shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:37<00:00, 148.35it/s]


(5568, 20, 384)

Baseline BiLSTM

In [7]:
num_epochs = 100
batch=64
hidden_dim_sizes = [200]#[100, 200, 300]
num_layers = 1
bidirectional = True
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-3]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience =4

history length = 20

In [8]:
size = 20
bilstm_history_20_kfold, best_bilstm_history_20_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    path_indices=None,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:40<00:00, 139.19it/s]





100%|██████████| 1/1 [03:04<00:00, 184.62s/it]

100%|██████████| 1/1 [03:49<00:00, 229.03s/it]


In [9]:
bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,batch_size,model_id
0,focal,0.512869,0.512570,"[0.5246511627906977, 0.5004887585532747]",0.219530,0.722420,0.671098,"[0.8010204081632653, 0.5411764705882354]",20,1,True,200,0.2,0.001,0,2,False,64,0
0,focal,0.526215,0.526200,"[0.5289099526066351, 0.5234899328859062]",0.294454,0.701068,0.662491,"[0.776595744680851, 0.5483870967741935]",20,1,True,200,0.2,0.001,1,2,False,64,0
0,focal,0.550048,0.548982,"[0.527054108216433, 0.5709090909090908]",0.356590,0.725979,0.678485,"[0.8020565552699229, 0.5549132947976878]",20,1,True,200,0.2,0.001,12,2,False,64,0
0,focal,0.548141,0.545294,"[0.5812720848056536, 0.5093167701863354]",0.451600,0.718861,0.670134,"[0.7969151670951157, 0.5433526011560693]",20,1,True,200,0.2,0.001,123,2,False,64,0
0,focal,0.545281,0.542058,"[0.5036420395421437, 0.5804749340369393]",0.451641,0.690391,0.660010,"[0.7616438356164383, 0.5583756345177665]",20,1,True,200,0.2,0.001,1234,2,False,64,0


In [10]:
best_bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,batch_size
0,focal,0.512869,0.512570,"[0.5246511627906977, 0.5004887585532747]",0.219530,0.722420,0.671098,"[0.8010204081632653, 0.5411764705882354]",1,True,200,0.2,0.001,0,2,False,64
0,focal,0.526215,0.526200,"[0.5289099526066351, 0.5234899328859062]",0.294454,0.701068,0.662491,"[0.776595744680851, 0.5483870967741935]",1,True,200,0.2,0.001,1,2,False,64
0,focal,0.550048,0.548982,"[0.527054108216433, 0.5709090909090908]",0.356590,0.725979,0.678485,"[0.8020565552699229, 0.5549132947976878]",1,True,200,0.2,0.001,12,2,False,64
0,focal,0.548141,0.545294,"[0.5812720848056536, 0.5093167701863354]",0.451600,0.718861,0.670134,"[0.7969151670951157, 0.5433526011560693]",1,True,200,0.2,0.001,123,2,False,64
0,focal,0.545281,0.542058,"[0.5036420395421437, 0.5804749340369393]",0.451641,0.690391,0.660010,"[0.7616438356164383, 0.5583756345177665]",1,True,200,0.2,0.001,1234,2,False,64


In [11]:
best_bilstm_history_20_kfold["f1"].mean()

0.535020883453311

In [12]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

array([0.53310587, 0.5369359 ])